# Access DB2 on Cloud using Python

Import the ibm_db Python library

In [1]:
import ibm_db

Identify the database connection credentials

In [2]:
#Replace the placeholder values with your actual Db2 hostname, username, and password:
dsn_hostname = "98538591-7217-4024-b027-8baa776ffad1.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud" # e.g.: "54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud"
dsn_uid = "xjv39764"        # e.g. "abc12345"
dsn_pwd = "***"      # e.g. "7dBZ3wWt9XN6$o0J"

dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB"            # e.g. "BLUDB"
dsn_port = "30875"                # e.g. "32733" 30875
dsn_protocol = "TCPIP"            # i.e. "TCPIP"
dsn_security = "SSL"              #i.e. "SSL"

Create the database connection

In [3]:
#Create database connection
#DO NOT MODIFY THIS CELL. Just RUN it with Shift + Enter
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};"
    "SECURITY={7};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd,dsn_security)

try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg() )


Connected to database:  BLUDB as user:  xjv39764 on host:  98538591-7217-4024-b027-8baa776ffad1.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud


Create a table in the database

In [4]:
#Lets first drop the table INSTRUCTOR in case it exists from a previous attempt
dropQuery = "drop table INSTRUCTOR"

#Now execute the drop statment
dropStmt = ibm_db.exec_immediate(conn, dropQuery)

In [5]:
#Construct the Create Table DDL statement - replace the ... with rest of the statement
createQuery = "create table INSTRUCTOR(id INTEGER PRIMARY KEY NOT NULL, \
                fname varchar(20), lname varchar(20), city varchar(20), ccode char(2))"

#Now fill in the name of the method and execute the statement
createStmt = ibm_db.exec_immediate(conn, createQuery)

Insert data into the table

In [6]:
#Construct the query - replace ... with the insert statement
insertQuery = "INSERT INTO Instructor(id, fname, lname, city, ccode) \
                VALUES(1, 'Rav', 'Ahuja', 'TORONTO', 'CA');"

#execute the insert statement
insertStmt = ibm_db.exec_immediate(conn, insertQuery)

In [7]:
insertQuery = "INSERT INTO Instructor \
            values (2, 'Raul', 'Chong', 'Markham', 'CA'), \
            (3, 'Hima', 'Vasudevan', 'Chicago', 'US');"

#execute the insert statement
insertStmt = ibm_db.exec_immediate(conn, insertQuery)

Query data in the table

In [8]:
#Construct the query that retrieves all rows from the INSTRUCTOR table
selectQuery = "select * from INSTRUCTOR"

#Execute the statement
selectStmt = ibm_db.exec_immediate(conn, selectQuery)

In [9]:
#Fetch the Dictionary (for the first row only) - replace ... with your code
ibm_db.fetch_both(selectStmt)

{'ID': 1,
 0: 1,
 'FNAME': 'Rav',
 1: 'Rav',
 'LNAME': 'Ahuja',
 2: 'Ahuja',
 'CITY': 'TORONTO',
 3: 'TORONTO',
 'CCODE': 'CA',
 4: 'CA'}

In [10]:
#Fetch the rest of the rows and print the ID and FNAME for those rows
while ibm_db.fetch_row(selectStmt) != False:
   print (" ID:",  ibm_db.result(selectStmt, 0),
          " FNAME:", ibm_db.result(selectStmt, "FNAME"),
          ibm_db.result(selectStmt, "FNAME"))

SystemError: Negative size passed to PyUnicode_New

In [11]:
selectQuery = "UPDATE INSTRUCTOR SET city='MOOSETOWN' WHERE fname='Rav';"
selectStmt = ibm_db.exec_immediate(conn, selectQuery)

Retrieve data into Pandas

In [12]:
import pandas
import ibm_db_dbi

In [13]:
#connection for pandas
pconn = ibm_db_dbi.Connection(conn)

In [14]:
#query statement to retrieve all rows in INSTRUCTOR table
selectQuery = "select * from INSTRUCTOR"

#retrieve the query results into a pandas dataframe
pdf = pandas.read_sql(selectQuery, pconn)

#print just the LNAME for first row in the pandas data frame
pdf.LNAME[0]

'Ahuja'

In [15]:
#print the entire data frame
pdf

,ID,FNAME,LNAME,CITY,CCODE
0,1,Rav,Ahuja,MOOSETOWN,CA
1,2,Raul,Chong,Markham,CA
2,3,Hima,Vasudevan,Chicago,US


In [16]:
pdf.shape

(3, 5)

Close the Connection

In [17]:
ibm_db.close(conn)

True